# Group 12 - HUM-274: Milestone 2

In [ ]:
from math import fabs
from music21 import midi, note, stream, instrument, meter

### 4) Adding complexity back

The previous part sees the deconstruction of the piece down to urban noises. This part stalls a little bit in this realm
of noise by injecting some rhythmic complexity first through a carefully chosen L-System, which we will reuse later.

Indeed, the L-System abstraction is really powerful as you can easily construct complex phenomena with varying effects.
First, here's our general purpose L-System implementation:

In [ ]:
class Rule:
    """
    Encodes L Systems' replacement rules.
    """

    def __init__(self, base: str, replacement: str):
        """
        :param base: string,
        :param replacement: string
        """
        self.base = base
        self.replacement = replacement


class LSystem:
    """
    L System's functionality.
    """

    def __init__(self, *rules):
        """
        An L System = a set of rules

        :param rules: List[Rule]
        """
        self.rules = {}
        [self.rules.update({rule.base: rule.replacement}) for rule in rules]

    def replace(self, base, show_mode: bool = False):
        """
        The brains of an L System, doing the simple task of applying each rule recursively

        :param base: some base string
        :param show_mode: boolean deciding whether to put separating brackets or not between rule applications
        :return: result of applications of all replacement rules on base string
        """
        new_string = ""
        for c in base:
            if c in self.rules:
                if show_mode:
                    new_string += "<"
                new_string += self.rules.get(c)
                if show_mode:
                    new_string += ">"
            else:
                new_string += c
        return new_string

    def run(self, initial: str, nb_iterations: int, show_mode: bool = False):
        """
        Applies the replace method nb_iterations times with base string initial

        :param initial: first base string
        :param nb_iterations: how many times rules replacements should occur
        :param show_mode: boolean deciding whether to put separating brackets or not between rule applications
        :return: last result of rule applications
        """
        string = initial
        for i in range(nb_iterations):
            string = self.replace(string, show_mode)
        return string

This abstraction produces strings which are consequently mapped to musical elements. This is where a human can have
control over the L-System. See, for example, this function mapping L-System sequences to durations.

In [ ]:
def sequence_from_string_complex(string: str):
    """
    To use with chars: A, B, C, D, E, F, +, -, [, ]
    A: half note
    B: quarter
    C: eighth
    D: sixteenth
    E: triplet
    F: quintuplet
    +: add previous and next duration
    -: make previous note a rest (value: -1 * duration of rest)
    [: extend previous duration by 50%
    ]: divide previous duration by 2
    :param string: input string
    :return: sequence of durations (ints)
    """

    def char_to_duration(c: str, tb: list):
        if c == 'A':
            tb.append(2)
        elif c == 'B':
            tb.append(1)
        elif c == 'C':
            tb.append(1 / 2)
        elif c == 'D':
            tb.append(1 / 4)
        elif c == 'E':
            tb.append(1 / 3)
        elif c == 'F':
            tb.append(1 / 5)
        elif c == '[':
            if len(tb) > 0:
                tb[-1] = tb[-1] + 0.5 * tb[-1]
        elif c == ']':
            if len(tb) > 0:
                if tb[-1] > float(1.0 / 1024):
                    tb[-1] = tb[-1] / 2
        elif c == '-':
            if len(tb) > 0:
                tb[-1] = -tb[-1]
        return tb[-1]

    def is_duration_char(c: str):
        return c in ['A', 'B', 'C', 'D', 'E', 'F']

    str_arr = [c for c in string]
    tab = []
    while not len(str_arr) == 0:
        nb_chars_read = 1

        if str_arr[0] == '+' and len(str_arr) >= 2:
            nb_chars_read = 2
            if len(tab) > 0 and is_duration_char(str_arr[1]):
                old_read = tab[-1]
                new_read = char_to_duration(str_arr[1], tab)
                new_dur = fabs(old_read) + fabs(new_read)
                if old_read < 0:
                    new_dur = -new_dur
                tab[-2] = new_dur
                tab = tab[:-1]

        else:
            char_to_duration(str_arr[0], tab)

        str_arr = str_arr[nb_chars_read:]  # remove chars read
    return tab

As we only care, in this section, about rhythmic quantities, we'll go ahead and use the helper functions given in the
second notebook of the course:

In [ ]:
def quarter_length():
    return 1/ 4

def create_percussion(instru=instrument.Woodblock(), time_sig=None):
    # Initialize a percussion stream with Woodblock timbre
    # If time signature is None, no measure splits
    if time_sig is None:
        drum_part = stream.Measure()
    else:
        drum_part = stream.Stream()
        drum_part.timeSignature = meter.TimeSignature(time_sig)

    drum_part.insert(0, instru)
    return drum_part


def append_event(duration, original_stream, rest=False, pitch='C4'):
    # Returns a new_stream obtained by appending a rhythmical event or a rest of given duration to the original_stream
    new_stream = original_stream
    if rest:
        new_stream.append(note.Rest(quarterLength=duration * (4 * quarter_length())))
    else:
        new_stream.append(note.Note(pitch, quarterLength=duration * (4 * quarter_length())))
    return new_stream


def rhythm_from_sequence(durations, instru=instrument.Woodblock(), time_sig=None, pitch='C4', rhythm=None):
    # Generate rhythmic stream from a list of durations. Rests are indicated by specifying a duration as a string
    if rhythm is None:
        # pass an existing stream 'rhythm' to append the durations, otherwise a new one will be created
        rhythm = create_percussion(instru, time_sig)
    for dur in durations:
        is_rest = False
        if dur != 0:
            if dur < 0:
                # if duration is given as a string, interpret as rest and turn string into a numerical value
                is_rest = True
                dur = -dur

            rhythm = append_event(dur, rhythm, rest=is_rest, pitch=pitch)
    return rhythm


def play(score):
    # Shortcut to play a stream
    midi.realtime.StreamPlayer(score).play()


Last but not least, let's put the pieces together! Let's define some rules and see what comes out.

In [ ]:
def rules_complex():
    rule_a = Rule("A", "BB[F+E-D+A]FF")
    rule_b = Rule("B", "D[E]-C""D[E]-C")
    rule_c = Rule("C", "CD+C-CF-")
    rule_d = Rule("D", "AE+[D-D]D+ED")
    rule_e = Rule("E", "[E+-]B")
    rule_f = Rule("F", "F+[-B]B[[A]EF]")
    return LSystem(rule_a, rule_b, rule_c, rule_d, rule_e, rule_f)


def initial_complex():
    return "F]AEE-B"


def chars_complex():
    return ["A", "B", "C", "D", "E", "F"]


def run_complex_for(n, show_mode=False):
    return rules_complex().run(initial_complex(), n, show_mode)

rhythm = rhythm_from_sequence(run_complex_for(40))
play(rhythm)

In [ ]:
rhythm.show()

We can now freely add melodic values to these durations.

### 5) Convergence after chaos
The second to last step of the composition sees a reconvergence to the main theme. Rhythm and melody converge separately
to a theme resembling that of the *Boléro*